# Generated flight route analysis
- Min, Max, Avg, mean, median distance
- Sectors analysis:
    - Example: VV-TSN VV-LATHA VV-NIXUP VV-CN WS-ESPOB WS-ENREP WS-VEPLI WM-EGOLO WM-ROBMO WM-VMR WS-PU20 WS-VTK -> unique VV4WS2WM2WS1 -> count unique
                                            - > VVWSWMWS -> count

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import groupby

## Load dataframe

In [2]:
PATH = "synthetic_flightplan.csv"
df = pd.read_csv(PATH)
print(df.head())

  origin destination  min_dist_origin  min_dist_dest  route_distances  \
0    PHS         DMK         1.032578       1.305384       174.521034   
1    PHS         DMK         1.032578       1.305384       309.726818   
2    PHS         DMK         1.032578       1.305384       310.535807   
3    PHS         DMK         1.032578       1.305384       347.363798   
4    PHS         DMK         1.032578       1.305384       348.172787   

   total_distances                                              route  
0       176.858995                       VT-PSL VT-PEBLI VT-TL VT-BKK  
1       312.064780  VT-PSL VT-PEBLI VT-TL VT-KRT VT-UBLOD VT-MALKI...  
2       312.873769  VT-PSL VT-PEBLI VT-TL VT-KRT VT-UBLOD VT-PIPOB...  
3       349.701760  VT-PSL VT-GOKON VT-PCB VT-CMP VT-KKN VT-UBLOD ...  
4       350.510749  VT-PSL VT-GOKON VT-PCB VT-CMP VT-KKN VT-UBLOD ...  


## Descriptive flight route analysis

In [3]:
df.total_distances.describe()

count    145751.000000
mean       1405.501531
std         956.220606
min           0.213919
25%         745.155369
50%        1122.979553
75%        1760.735573
max        7020.821435
Name: total_distances, dtype: float64

In [4]:
unique_od_pairs = df.groupby(['origin', 'destination'])
unique_od_pairs.total_distances.describe().sort_values(by='count', ascending=False)

count         mean         std          min          25%  \
origin destination                                                             
ZVK    VTE          100.0   711.049835   67.028404   441.702801   709.722618   
ZBY    VTE          100.0   936.358872   93.445092   493.599846   915.447560   
ADL    DPS          100.0  3122.438555  128.087176  2095.112988  3101.557728   
       KUL          100.0  3206.537289   46.486918  3119.825384  3168.396937   
       SIN          100.0  3195.251112   62.235345  2981.116012  3171.023839   
...                   ...          ...         ...          ...          ...   
UPG    MOH            1.0   218.536137         NaN   218.536137   218.536137   
       KXB            1.0   175.720693         NaN   175.720693   175.720693   
       KDI            1.0   188.752760         NaN   188.752760   188.752760   
AMQ    SXK            1.0   334.078128         NaN   334.078128   334.078128   
ZRI    DJJ            1.0   889.622013         NaN   889.622013   889.622013   

                            50%          75%          max  
origin destination                                         
ZVK    VTE           734.838413   754.068944   766.671101  
ZBY    VTE           967.126492   993.182547  1018.957023  
ADL    DPS          3149.459097  3187.706189  3238.107157  
       KUL          3196.698608  3253.408040  3274.866804  
       SIN          3210.899799  3243.675527  3264.405901  
...                         ...          ...          ...  
UPG    MOH           218.536137   218.536137   218.536137  
       KXB           175.720693   175.720693   175.720693  
       KDI           188.752760   188.752760   188.752760  
AMQ    SXK           334.078128   334.078128   334.078128  
ZRI    DJJ           889.622013   889.622013   889.622013  

[1724 rows x 8 columns]

In [5]:
# Top 10 od_pairs with the highest standard deviation of total_distances
k=10
top_k_od_pairs = unique_od_pairs.total_distances.std().sort_values(ascending=False).head(k)
for i, (od_pair, std) in enumerate(top_k_od_pairs.items()):
    print(f"{i+1}. {od_pair[0]} {od_pair[1]}: {std:.2f}nm")

1. BMU UPG: 825.07nm
2. SOQ BXB: 690.19nm
3. KOE LWE: 688.24nm
4. KOE MOF: 688.24nm
5. BPN PLW: 682.19nm
6. KOE ABU: 652.39nm
7. MKQ DJJ: 617.27nm
8. NRE AMQ: 604.33nm
9. NYW HOX: 600.82nm
10. NYU HEH: 552.65nm


## Sector analysis

In [6]:

def route_to_sector_unqiue_count(route: str) -> str:
    """ 
    Example:
    "VV-TSN VV-LATHA VV-NIXUP VV-CN WS-ESPOB WS-ENREP WS-VEPLI WM-EGOLO WM-ROBMO WM-VMR WS-PU20 WS-VTK" -> "VV4WS3WM3WS2"
    """
    # Split the route into segments; handle empty input
    segments = route.split()
    if not segments:
        return ""
    
    # Define a key function to extract the two-letter code from each segment
    def key_func(segment):
        return segment.split('-')[0]
    
    # Initialize result list to store code-count pairs
    result = []
    
    # Group consecutive segments by their code and count each group
    for code, group in groupby(segments, key=key_func):
        count = sum(1 for _ in group)  # Count the number of segments in the group
        result.append(code + str(count))  # Append code followed by count
    
    # Join all parts into a single string without separators
    return '-'.join(result)

def route_to_sector_unqiue(route: str) -> str:
    # Split the route into segments
    segments = route.split()
    # Handle empty input
    if not segments:
        return ""
    
    # Extract the two-letter code from each segment
    codes = [segment.split('-')[0] for segment in segments]
    
    # Remove consecutive duplicates using groupby
    unique_codes = [key for key, _ in groupby(codes)]
    
    # Join the unique codes with hyphens
    return '-'.join(unique_codes)

### Uniquely sectors counting analysis

Example route: "VV-TSN VV-LATHA VV-NIXUP VV-CN WS-ESPOB WS-ENREP WS-VEPLI WM-EGOLO WM-ROBMO WM-VMR WS-PU20 WS-VTK" -> "VV4WS3WM3WS2"

Then execute group by with "VV4WS3WM3WS2", after that analyze the total distance

In [7]:
tmp_a = df.copy()
unique_count_sectors_df = tmp_a.route.apply(route_to_sector_unqiue_count)
tmp_a['unique_count_sectors'] = unique_count_sectors_df
tmp_a.tail()

,origin,destination,min_dist_origin,min_dist_dest,route_distances,total_distances,route,unique_count_sectors
145746,OIR,HKD,0.471177,21.513870,857.914510,879.899557,RJ-ORE RJ-MAIKA RJ-ESASI RJ-PATRA RJ-HWE RJ-TA...,RJ26
145747,OIR,HKD,25.485857,21.601246,851.398901,898.486005,RJ-MAIKA RJ-ESASI RJ-PATRA RJ-HWE RJ-ARIKA RJ-...,RJ25
145748,OIR,HKD,0.471177,21.601246,876.415758,898.488181,RJ-ORE RJ-MAIKA RJ-ESASI RJ-PATRA RJ-HWE RJ-AR...,RJ26
145749,OIR,HKD,25.485857,21.601246,853.255785,900.342888,RJ-MAIKA RJ-ESASI RJ-PATRA RJ-HWE RJ-TAPPI RJ-...,RJ26
145750,OIR,HKD,0.471177,21.601246,878.272641,900.345064,RJ-ORE RJ-MAIKA RJ-ESASI RJ-PATRA RJ-HWE RJ-TA...,RJ27


In [8]:
# Total flight route analysis by unique_count_sectors
tmp_a.groupby([ 'origin', 'destination', 'unique_count_sectors']).total_distances.describe().sort_values(by='min', ascending=False)

count  \
origin destination unique_count_sectors                                 
AKL    PVG         NZ2-YB12-WA3-RP6-WB7-WS3-VV29-ZG12-ZS13        1.0   
                   NZ2-YB12-WA3-RP12-WB7-WS3-VV27-ZG12-ZS13       2.0   
                   NZ2-YB16-WA4-WB9-WS3-VV29-ZG12-ZS13            1.0   
                   NZ2-YB16-WA2-YB1-WA4-WB3-WS3-VV30-ZG12-ZS13    1.0   
                   NZ2-YB12-WA3-RP22-VV6-ZJ1-VV18-ZG12-ZS13       1.0   
...                                                               ...   
MNL    MNL         RP2                                            7.0   
TKN    OKE         RJ1                                            1.0   
OKE    TKN         RJ1                                            1.0   
MUR    MYY         WB1                                            2.0   
MNL    MNL         RP1                                            1.0   

                                                                       mean  \
origin destination unique_count_sectors                                       
AKL    PVG         NZ2-YB12-WA3-RP6-WB7-WS3-VV29-ZG12-ZS13      7017.538313   
                   NZ2-YB12-WA3-RP12-WB7-WS3-VV27-ZG12-ZS13     7018.251744   
                   NZ2-YB16-WA4-WB9-WS3-VV29-ZG12-ZS13          7015.526533   
                   NZ2-YB16-WA2-YB1-WA4-WB3-WS3-VV30-ZG12-ZS13  7012.444213   
                   NZ2-YB12-WA3-RP22-VV6-ZJ1-VV18-ZG12-ZS13     7011.053937   
...                                                                     ...   
MNL    MNL         RP2                                            36.640543   
TKN    OKE         RJ1                                            26.546986   
OKE    TKN         RJ1                                            26.546986   
MUR    MYY         WB1                                            23.142055   
MNL    MNL         RP1                                             0.213919   

                                                                     std  \
origin destination unique_count_sectors                                    
AKL    PVG         NZ2-YB12-WA3-RP6-WB7-WS3-VV29-ZG12-ZS13           NaN   
                   NZ2-YB12-WA3-RP12-WB7-WS3-VV27-ZG12-ZS13     3.634091   
                   NZ2-YB16-WA4-WB9-WS3-VV29-ZG12-ZS13               NaN   
                   NZ2-YB16-WA2-YB1-WA4-WB3-WS3-VV30-ZG12-ZS13       NaN   
                   NZ2-YB12-WA3-RP22-VV6-ZJ1-VV18-ZG12-ZS13          NaN   
...                                                                  ...   
MNL    MNL         RP2                                          5.386876   
TKN    OKE         RJ1                                               NaN   
OKE    TKN         RJ1                                               NaN   
MUR    MYY         WB1                                          0.136173   
MNL    MNL         RP1                                               NaN   

                                                                        min  \
origin destination unique_count_sectors                                       
AKL    PVG         NZ2-YB12-WA3-RP6-WB7-WS3-VV29-ZG12-ZS13      7017.538313   
                   NZ2-YB12-WA3-RP12-WB7-WS3-VV27-ZG12-ZS13     7015.682054   
                   NZ2-YB16-WA4-WB9-WS3-VV29-ZG12-ZS13          7015.526533   
                   NZ2-YB16-WA2-YB1-WA4-WB3-WS3-VV30-ZG12-ZS13  7012.444213   
                   NZ2-YB12-WA3-RP22-VV6-ZJ1-VV18-ZG12-ZS13     7011.053937   
...                                                                     ...   
MNL    MNL         RP2                                            29.110724   
TKN    OKE         RJ1                                            26.546986   
OKE    TKN         RJ1                                            26.546986   
MUR    MYY         WB1                                            23.045766   
MNL    MNL         RP1                                             0.213919   

                                                                     

In [9]:
# Top k unique_count_sectors with the highest standard deviation of total_distances
k=10
top_k_unique_count_sectors = tmp_a.groupby([ 'origin', 'destination', 'unique_count_sectors']).total_distances.std().sort_values(ascending=False).head(k)
for i, (unique_count_sectors, std) in enumerate(top_k_unique_count_sectors.items()):
    print(f"{i+1}. {unique_count_sectors}: {std:.2f}nm")
    

1. ('WMX', 'DJJ', 'WA20'): 543.42nm
2. ('BIK', 'DJJ', 'WA24'): 530.42nm
3. ('DJJ', 'WMX', 'WA15'): 518.22nm
4. ('MKW', 'SOQ', 'WA16'): 503.97nm
5. ('DJJ', 'MKQ', 'WA16'): 478.35nm
6. ('NBX', 'DJJ', 'WA24'): 472.46nm
7. ('GTO', 'UPG', 'WA10'): 446.43nm
8. ('MDC', 'UPG', 'WA8'): 446.43nm
9. ('LUW', 'UPG', 'WA10'): 446.43nm
10. ('SOQ', 'TIM', 'WA11'): 435.51nm


In [16]:
route_count_unique_sectors = tmp_a.groupby([ 'origin', 'destination', 'unique_count_sectors']).total_distances.describe()

data = {}

for _, row in route_count_unique_sectors.iterrows():
    route = row.name[2]
    count = row['count']

    for sector_xx in route.split('-'):
        sector = sector_xx[:2]
        number = int(sector_xx[2:])
        if sector not in data:
            data[sector] = 0
        data[sector] += count * number

# Top affected sectors
k=10
top_k_affected_sectors = pd.Series(data).sort_values(ascending=False).head(k)
for i, (sector, count) in enumerate(top_k_affected_sectors.items()):
    print(f"{i+1}. {sector}: {count:.0f}")

1. VT: 505686
2. VV: 479124
3. RP: 453050
4. WM: 319098
5. WI: 318398
6. RJ: 314400
7. WA: 257039
8. RC: 210356
9. WS: 170339
10. WB: 121461


### Unique sector analysis

Example route: "VV-TSN VV-LATHA VV-NIXUP VV-CN WS-ESPOB WS-ENREP WS-VEPLI WM-EGOLO WM-ROBMO WM-VMR WS-PU20 WS-VTK" -> "VVWSWMWS"

Then execute group by with "VVWSWMWS", after that analyze the total distance

In [17]:
tmp_b = df.copy()
unique_sectors_df = tmp_b.route.apply(route_to_sector_unqiue)
tmp_b['unique_sectors'] = unique_sectors_df
tmp_b.head()

,origin,destination,min_dist_origin,min_dist_dest,route_distances,total_distances,route,unique_sectors
0,PHS,DMK,1.032578,1.305384,174.521034,176.858995,VT-PSL VT-PEBLI VT-TL VT-BKK,VT
1,PHS,DMK,1.032578,1.305384,309.726818,312.064780,VT-PSL VT-PEBLI VT-TL VT-KRT VT-UBLOD VT-MALKI...,VT
2,PHS,DMK,1.032578,1.305384,310.535807,312.873769,VT-PSL VT-PEBLI VT-TL VT-KRT VT-UBLOD VT-PIPOB...,VT
3,PHS,DMK,1.032578,1.305384,347.363798,349.701760,VT-PSL VT-GOKON VT-PCB VT-CMP VT-KKN VT-UBLOD ...,VT
4,PHS,DMK,1.032578,1.305384,348.172787,350.510749,VT-PSL VT-GOKON VT-PCB VT-CMP VT-KKN VT-UBLOD ...,VT


In [22]:
# Total flight route analysis by unique_sectors
unique_sectors_df = tmp_b.groupby(['origin','destination','unique_sectors']).total_distances.describe()
unique_sectors_df

count         mean         std  \
origin destination unique_sectors                                         
ABU    KOE         WA                     10.0  1456.796706  188.018993   
ADL    DPS         YM-WI-WA               58.0  3123.948769   86.892421   
                   YM-WI-WA-WI-WA         17.0  3137.471213   60.968214   
                   YM-YB-WA                1.0  2095.112988         NaN   
                   YM-YB-YM-WI-WA         15.0  3114.839626   15.219437   
...                                        ...          ...         ...   
ZBY    VTE         VL-VV-ZJ-VV-VL-VT-VL    2.0   997.375616   14.289567   
ZRI    BIK         WA                      4.0   922.640867    7.139069   
       DJJ         WA                      1.0   889.622013         NaN   
ZVK    VTE         VL-VT-VL              100.0   711.049835   67.028404   
ZYI    WUH         ZP-ZG-ZH                3.0   441.744201   18.247931   

                                                 min          25%  \
origin destination unique_sectors                                   
ABU    KOE         WA                    1182.326011  1325.775653   
ADL    DPS         YM-WI-WA              2978.766117  3044.060350   
                   YM-WI-WA-WI-WA        3077.259869  3084.899960   
                   YM-YB-WA              2095.112988  2095.112988   
                   YM-YB-YM-WI-WA        3106.684290  3107.324098   
...                                              ...          ...   
ZBY    VTE         VL-VV-ZJ-VV-VL-VT-VL   987.271367   992.323491   
ZRI    BIK         WA                     916.315122   916.532362   
       DJJ         WA                     889.622013   889.622013   
ZVK    VTE         VL-VT-VL               441.702801   709.722618   
ZYI    WUH         ZP-ZG-ZH               426.994819   431.540937   

                                                 50%          75%          max  
origin destination unique_sectors                                               
ABU    KOE         WA                    1485.057654  1623.238884  1690.408333  
ADL    DPS         YM-WI-WA              3159.093083  3168.247153  3238.107157  
                   YM-WI-WA-WI-WA        3086.178043  3187.706347  3232.241055  
                   YM-YB-WA              2095.112988  2095.112988  2095.112988  
                   YM-YB-YM-WI-WA        3107.963000  3107.964012  3146.596446  
...                                              ...          ...          ...  
ZBY    VTE         VL-VV-ZJ-VV-VL-VT-VL   997.375616  1002.427741  1007.479866  
ZRI    BIK         WA                     922.640867   928.749373   928.966613  
       DJJ         WA                     889.622013   889.622013   889.622013  
ZVK    VTE         VL-VT-VL               734.838413   754.068944   766.671101  
ZYI    WUH         ZP-ZG-ZH               436.087056   449.118892   462.150729  

[5251 rows x 8 columns]